# Import Library

In [ ]:
import mne
import numpy as np
import os
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs, apply_inverse
from mne.datasets import fetch_fsaverage
import scipy.io
from scipy.io import loadmat
from scipy.spatial import Delaunay
import PIL
from PIL import Image
import datetime
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense, Flatten, Concatenate, BatchNormalization, Dropout, Input
from keras.layers.merge import concatenate
from tensorflow.keras.optimizers import Adam
# Load the TensorBoard notebook extension
%load_ext tensorboard
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, KFold
import gc

%matplotlib inline
#%matplotlib qt

DIRECTORY_PATH = os.getcwd()
EXTERNAL_STORAGE_PATH = "D:\Motor Imagery"
n_splits = 5

# Preprocess Data

In [ ]:
img = nib.load("/Users/ivanl/Downloads/MRIcron_windows/MRIcron/Resources/templates/brodmann.nii.gz")

brodmann_data = img.get_fdata()
# Area 17 – Primary visual cortex (V1)
brodmann_visual = None
selected_area = [17]
for area in selected_area:
    if brodmann_visual is None:
        brodmann_visual = brodmann_data.reshape(-1) == area
    else:
        brodmann_visual += brodmann_data.reshape(-1) == area

print(brodmann_visual)
print("brodmann template shape: " + str(brodmann_data.shape))
print("chosen points: " + str(np.sum(brodmann_visual)))

shape, affine = img.shape[:3], img.affine
coords = np.array(np.meshgrid(*(range(i) for i in shape), indexing='ij'))
coords = np.rollaxis(coords, 0, len(shape) + 1)
mm_coords = nib.affines.apply_affine(affine, coords)

def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

my_left_points = None
my_right_points = None

In [ ]:
# cd to google drive
os.chdir("G:")

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
mne_subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

source = mne.read_source_spaces(src)
left = source[0]
right = source[1]
left_pos = left["rr"][left["inuse"]==1]
right_pos = right["rr"][right["inuse"]==1]
                        
transformation = mne.read_trans(op.join(fs_dir, "bem", "fsaverage-trans.fif"))

save_path = op.join(os.getcwd(), "Shared drives", "Motor Imagery", "Source Estimate")

In [ ]:
"""
create mne epochs data structure from numpy array
merge training and evaluation data
"""
def create_epochs(data_path):
    subjects_data = {}
    files = os.listdir(data_path)
    for file in files:
        # load data
        data = loadmat(op.join(data_path, file))
        sampling_freq = data['Fs']
        labels = data['categoryLabels'].reshape(-1)     # trials
        epochs = data['X_3D']                           # no of channels, time, trials
        epochs = np.moveaxis(epochs, [0, 1], [1, 2])    # trials, no of channels, time

        # create info
        GSN_128 = mne.channels.make_standard_montage('GSN-HydroCel-128')
        ch_names = GSN_128.ch_names[:124]
        ch_types = ['eeg'] * 124
        info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
        info.set_montage(GSN_128)

        epochs = mne.EpochsArray(epochs, info, verbose=False)
        subjects_data[file[:-4]] = {}
        subjects_data[file[:-4]]["epochs"] = epochs
        subjects_data[file[:-4]]["labels"] = labels

    return subjects_data

In [ ]:
"""
Create source activity and reconstructed eeg respectively for each subject

For each subject, there are six events in total, i.e. 
(1=Human Body; 2=Human Face; 3=Animal Body; 4=Animal Face; 5=Fruit Vegetable; 6=Inanimate Object) 
Split these data into train and test set using kfold
Compute the noise covariance matrix on train set and apply it to test set
Create source activity (only motor region) first by applying an inverse operator to the epochs 
Create reconstructed eeg by applying a forward operator to the source activity acquired earlier
Save both these files to disk
"""
def apply_inverse_and_forward_kfold(epochs, n_splits=5, save_inverse=True, save_forward=True):
    global my_left_points, my_right_points
    
    for subject in epochs.keys():  
        X, Y = [], []
        info = None
        counter = 0
        
        X = epochs[subject]["epochs"].get_data()
        print(X.shape)
        Y = epochs[subject]["labels"]
        print(Y.shape)
        info = epochs[subject]["epochs"].info
        
        X = np.array(X)
        Y = np.array(Y)
        
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        for train_index, test_index in skf.split(X, Y):
            counter += 1
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            X_train = mne.EpochsArray(X_train, info, verbose=False)
            X_test = mne.EpochsArray(X_test, info, verbose=False)
            
            noise_cov = mne.compute_covariance(X_train, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
            fwd = mne.make_forward_solution(info, trans=trans, src=src,
                            bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1, verbose=False)
            fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True, verbose=False)
            leadfield = fwd_fixed['sol']['data']
            inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)
            
            method = "sLORETA"
            snr = 3.
            lambda2 = 1. / snr ** 2
            stc_train = apply_inverse_epochs(X_train, inverse_operator, lambda2,
                                          method=method, pick_ori="normal", verbose=True)
            
            # get motor region points (once)
            if my_left_points is None and my_right_points is None:
                my_source = stc_train[0]
                mni_lh = mne.vertex_to_mni(my_source.vertices[0], 0, mne_subject)
                #print(mni_lh.shape)
                mni_rh = mne.vertex_to_mni(my_source.vertices[1], 1, mne_subject)
                #print(mni_rh.shape)

                """
                fig = plt.figure(figsize=(8, 8))
                ax = fig.add_subplot(projection='3d')
                ax.scatter(mm_coords.reshape(-1, 3)[brodmann_visual][:, 0], mm_coords.reshape(-1, 3)[brodmann_visual][:, 1], mm_coords.reshape(-1, 3)[brodmann_visual][:, 2], s=15, marker='|')
                ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='_')
                ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
                ax.set_xlabel('X Label')
                ax.set_ylabel('Y Label')
                ax.set_zlabel('Z Label')
                plt.show()
                """

                my_left_points = in_hull(mni_lh, mm_coords.reshape(-1, 3)[brodmann_visual])
                my_right_points = in_hull(mni_rh, mm_coords.reshape(-1, 3)[brodmann_visual])

                mni_left_motor = mne.vertex_to_mni(my_source.vertices[0][my_left_points], 0, mne_subject)
                #print(mni_left_motor.shape)
                mni_right_motor = mne.vertex_to_mni(my_source.vertices[1][my_right_points], 1, mne_subject)
                #print(mni_right_motor.shape)

                """
                fig = plt.figure(figsize=(8, 8))
                ax = fig.add_subplot(projection='3d')
                ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
                ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
                ax.scatter(mni_left_motor[:, 0], mni_left_motor[:, 1], mni_left_motor[:, 2], s=15, marker='o')
                ax.scatter(mni_right_motor[:, 0], mni_right_motor[:, 1], mni_right_motor[:, 2], s=15, marker='^')
                ax.set_xlabel('X Label')
                ax.set_ylabel('Y Label')
                ax.set_zlabel('Z Label')
                plt.show()
                """
                
            #print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)
            #print(stc_train[0].data.shape)
            
            # train set
            # slice source activity data
            left_hemi_data = []
            right_hemi_data = []
            for source in stc_train:
                left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
                right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
            left_hemi_data = np.array(left_hemi_data)
            right_hemi_data = np.array(right_hemi_data)
            if save_inverse:
                source_activity_path = op.join(EXTERNAL_STORAGE_PATH, "data", "source activity", subject)
                if not op.exists(source_activity_path):
                    os.makedirs(source_activity_path)
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_train_X.npz"), data=np.append(left_hemi_data, right_hemi_data, axis=1))
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_train_Y.npz"), data=Y_train)
            # slice reconstructed eeg data
            reconstructed_eeg_data = []
            for source in stc_train:
                motor_source = np.zeros_like(source.data)
                motor_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
                motor_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
                motor_eeg = np.dot(leadfield, motor_source)
                reconstructed_eeg_data.append(motor_eeg)
            if save_forward:
                reconstructed_eeg_path = op.join(EXTERNAL_STORAGE_PATH, "data", "reconstructed eeg", subject)
                if not op.exists(reconstructed_eeg_path):
                    os.makedirs(reconstructed_eeg_path)
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_train_X.npz"), data=np.array(reconstructed_eeg_data))
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_train_Y.npz"), data=Y_train)
            
            del stc_train
            gc.collect()
            
            stc_test = apply_inverse_epochs(X_test, inverse_operator, lambda2,
                              method=method, pick_ori="normal", verbose=True)
            # test set
            # slice source activity data
            left_hemi_data = []
            right_hemi_data = []
            for source in stc_test:
                left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
                right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
            left_hemi_data = np.array(left_hemi_data)
            right_hemi_data = np.array(right_hemi_data)
            if save_inverse:
                source_activity_path = op.join(EXTERNAL_STORAGE_PATH, "data", "source activity", subject)
                if not op.exists(source_activity_path):
                    os.makedirs(source_activity_path)
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_test_X.npz"), data=np.append(left_hemi_data, right_hemi_data, axis=1))
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_test_Y.npz"), data=Y_test)
            # slice reconstructed eeg data
            reconstructed_eeg_data = []
            for source in stc_test:
                motor_source = np.zeros_like(source.data)
                motor_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
                motor_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
                motor_eeg = np.dot(leadfield, motor_source)
                reconstructed_eeg_data.append(motor_eeg)
            if save_forward:
                reconstructed_eeg_path = op.join(EXTERNAL_STORAGE_PATH, "data", "reconstructed eeg", subject)
                if not op.exists(reconstructed_eeg_path):
                    os.makedirs(reconstructed_eeg_path)
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_test_X.npz"), data=np.array(reconstructed_eeg_data))
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_test_Y.npz"), data=Y_test)
            
            del X_train, X_test, Y_train, Y_test
            del stc_test, reconstructed_eeg_data, left_hemi_data, right_hemi_data
            gc.collect()

In [ ]:
data_path = "Shared drives/Motor Imagery/Visual Dataset"
    
epochs = create_epochs(data_path)
print(epochs.keys())

In [ ]:
n_splits = 5

apply_inverse_and_forward_kfold(epochs, n_splits=n_splits, save_inverse=True, save_forward=True)